In [1]:
!pip install sentence-transformers
from sentence_transformers.util import cos_sim

import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util , InputExample
from torch.utils.data import DataLoader
from sentence_transformers import   SentenceTransformer, LoggingHandler, losses, models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b284bdedbf7cdde323778a4723b94acb05ff411b291ee7f941c671998362fc98
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
similarity_dataset = pd.read_csv('../input/quran-similar-versessemantic-similarity-search/pairs_quran_dataset.csv')
quran_dataset = pd.read_excel('../input/the-holy-quran/Dataset-Verse-by-Verse.xlsx')
all_quran = quran_dataset['ArabicText'].tolist()
input_text = "الحمد لله رب العالمين"

In [3]:
similarity_dataset

,premise,hypothesis,label
0,بسم الله الرحمن الرحيم,الحمد لله رب العالمين,2
1,بسم الله الرحمن الرحيم,الرحمن الرحيم,1
2,بسم الله الرحمن الرحيم,مالك يوم الدين,0
3,بسم الله الرحمن الرحيم,الم,0
4,بسم الله الرحمن الرحيم,الله لا إله إلا هو الحي القيوم,0
...,...,...,...
7674,لم يلد ولم يولد,وجعلوا بينه وبين الجنة نسبًا ۚ ولقد علمت الجنة...,2
7675,لم يلد ولم يولد,سبحان الله عما يصفون,1
7676,ولم يكن له كفوًا أحد,بديع السماوات والأرض ۖ أنى يكون له ولد ولم تكن...,2
7677,قل أعوذ برب الفلق,فالق الإصباح وجعل الليل سكنًا والشمس والقمر حس...,2


In [4]:
train_samples = []
for row in tqdm(similarity_dataset.itertuples()):
    train_samples.append(InputExample(texts=[row[1], row[2]], label=row[3]))

7679it [00:00, 403974.27it/s]


In [5]:
# create a dataloader
from torch.utils.data import DataLoader
batch_size = 8

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)

In [6]:
# create a sentence model
bert = models.Transformer('aubmindlab/bert-base-arabertv02')
pooling = models.Pooling(bert.get_word_embedding_dimension() , pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[bert, pooling])
model

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [7]:
loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)
epochs=1
warmup_steps= int(len(train_dataloader) * 0.1) #10% of train data for warm-up
model.fit(train_objectives=[(train_dataloader, loss)], epochs=epochs, warmup_steps=warmup_steps, output_path='output_of_bert/')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/960 [00:00<?, ?it/s]

In [8]:
all_quran = quran_dataset['ArabicText'].tolist()
all_quran_embeddings = model.encode(all_quran , batch_size=16) #calculated once

Batches:   0%|          | 0/390 [00:00<?, ?it/s]

In [9]:
sentence1 = "الحمدلله رب العالمين"
emb1 = model.encode(sentence1)
similarity_scores = cos_sim(emb1 , all_quran_embeddings)[0].numpy().tolist()
top_10_indecies = np.argsort(similarity_scores)[::-1][:11]
results = set(quran_dataset['ArabicText'][top_10_indecies].tolist())
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'إنه هو السميع العليم',
 'الحمد لله رب العالمين',
 'بسم الله الرحمن الرحيم',
 'ذو العرش المجيد',
 'سبحان الله عما يصفون',
 'سبحان ربك رب العزة عما يصفون',
 'فلله الحمد رب السماوات ورب الأرض رب العالمين',
 'فما ظنكم برب العالمين',
 'والحمد لله رب العالمين',
 'وسلام على المرسلين',
 'وما علينا إلا البلاغ المبين'}

In [10]:
s1 = model.encode(similarity_dataset['premise'] , batch_size=32)
s2 = model.encode(similarity_dataset['hypothesis'] , batch_size=32)

Batches:   0%|          | 0/240 [00:00<?, ?it/s]

Batches:   0%|          | 0/240 [00:00<?, ?it/s]

In [11]:
L=[]
for i in range(7679):
    L.append(cos_sim(s1[i] , s2[i])[0].item())

In [12]:
similarity_dataset['sim_scores'] = pd.Series(L)

In [13]:
TN = (similarity_dataset[similarity_dataset['label'] == 0]['sim_scores'] < 0.6).sum()

In [14]:
TP = (similarity_dataset[similarity_dataset['label'] == 2]['sim_scores'] > 0.6).sum() + (similarity_dataset[similarity_dataset['label'] == 1]['sim_scores'] > 0.6).sum()

In [15]:
FN = (similarity_dataset[similarity_dataset['label'] == 2]['sim_scores'] < 0.6).sum() + (similarity_dataset[similarity_dataset['label'] == 1]['sim_scores'] < 0.6).sum()
FP = (similarity_dataset[similarity_dataset['label'] == 0]['sim_scores'] > 0.6).sum()

In [16]:
# accuracy
print('Accuracy : ' , (TP + TN) / (TP + TN + FP + FN))
# precision
print('Precision : ' , TP / (TP + FP))
# recall
print('Recall : ', TP / (TP + FN))
# F1 score
print('F1 score : ',(2 * TP) / (2 * TP + FP + FN))

Accuracy :  0.8105222034119026
Precision :  0.9130838230745437
Recall :  0.8686185081653671
F1 score :  0.8902963130513458


In [17]:
similarity_dataset

,premise,hypothesis,label,sim_scores
0,بسم الله الرحمن الرحيم,الحمد لله رب العالمين,2,0.885365
1,بسم الله الرحمن الرحيم,الرحمن الرحيم,1,0.802723
2,بسم الله الرحمن الرحيم,مالك يوم الدين,0,0.687602
3,بسم الله الرحمن الرحيم,الم,0,0.646226
4,بسم الله الرحمن الرحيم,الله لا إله إلا هو الحي القيوم,0,0.757427
...,...,...,...,...
7674,لم يلد ولم يولد,وجعلوا بينه وبين الجنة نسبًا ۚ ولقد علمت الجنة...,2,0.555346
7675,لم يلد ولم يولد,سبحان الله عما يصفون,1,0.684509
7676,ولم يكن له كفوًا أحد,بديع السماوات والأرض ۖ أنى يكون له ولد ولم تكن...,2,0.552875
7677,قل أعوذ برب الفلق,فالق الإصباح وجعل الليل سكنًا والشمس والقمر حس...,2,0.599870


In [18]:
model.save_to_hub("my-new-model")

ValueError: You must login to the Hugging Face hub on this computer by typing `transformers-cli login`.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
?notebook_login